## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-03-04-42-53 +0000,nypost,Videos allegedly show ‘reckless’ NYC subway op...,https://nypost.com/2025/09/03/us-news/videos-a...
1,2025-09-03-04-35-18 +0000,nyt,China’s Military Spectacle,https://www.nytimes.com/2025/09/03/world/asia/...
2,2025-09-03-04-30-06 +0000,nyt,Putin’s Propaganda War,https://www.nytimes.com/2025/09/03/briefing/pu...
3,2025-09-03-04-15-08 +0000,nyt,Kim Jong-un Brings a His Daughter and Potentia...,https://www.nytimes.com/2025/09/02/world/asia/...
4,2025-09-03-04-10-01 +0000,bbc,'Lucky' Djokovic to face Alcaraz after battlin...,https://www.bbc.com/sport/tennis/articles/c4gz...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2332/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
58,trump,49
13,china,15
129,new,14
43,house,13
19,kim,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
233,2025-09-02-14-16-14 +0000,latimes,Trump deployment of military troops to Los Ang...,https://www.latimes.com/california/story/2025-...,104
190,2025-09-02-17-48-32 +0000,bbc,Trump's use of National Guard in Los Angeles w...,https://www.bbc.com/news/articles/c5ylyd9lkkqo...,104
65,2025-09-02-23-36-33 +0000,cbc,U.S. military kills 11 in strike on alleged dr...,https://www.cbc.ca/news/world/trump-venezuela-...,92
112,2025-09-02-21-04-00 +0000,wsj,Trump’s Use of Troops in Los Angeles Was Unlaw...,https://www.wsj.com/us-news/law/trumps-troop-d...,90
162,2025-09-02-19-04-50 +0000,nypost,‘Morning Joe’ host Joe Scarborough urges defia...,https://nypost.com/2025/09/02/us-news/joe-scar...,88


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
233,104,2025-09-02-14-16-14 +0000,latimes,Trump deployment of military troops to Los Ang...,https://www.latimes.com/california/story/2025-...
18,74,2025-09-03-03-14-25 +0000,wapo,"China’s Xi, flanked by Putin and Kim, shows of...",https://www.washingtonpost.com/world/2025/09/0...
60,45,2025-09-03-00-03-52 +0000,latimes,Cardi B wins civil assault trial brought by se...,https://www.latimes.com/entertainment-arts/sto...
71,40,2025-09-02-23-13-56 +0000,nypost,Jeffrey Epstein’s victims revealed ‘additional...,https://nypost.com/2025/09/02/us-news/jeffrey-...
27,38,2025-09-03-02-23-51 +0000,nypost,Florida man allegedly runs over foot model wit...,https://nypost.com/2025/09/02/us-news/florida-...
304,36,2025-09-02-08-23-36 +0000,nypost,"Sudan landslide kills over 1,000 people and wi...",https://nypost.com/2025/09/02/world-news/sudan...
38,36,2025-09-03-01-18-00 +0000,wsj,President Trump said he ordered a strike again...,https://www.wsj.com/politics/national-security...
205,34,2025-09-02-16-46-17 +0000,bbc,UK borrowing costs hit 27-year high adding to ...,https://www.bbc.com/news/articles/cy989njnq2wo...
84,34,2025-09-02-22-18-35 +0000,nypost,Six candidates from Germany’s rightwing AfD pa...,https://nypost.com/2025/09/02/world-news/six-c...
3,33,2025-09-03-04-15-08 +0000,nyt,Kim Jong-un Brings a His Daughter and Potentia...,https://www.nytimes.com/2025/09/02/world/asia/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
